In [5]:
load("../framework/instance_gen.sage")
load("../framework/geometry_twist.sage")
load("../framework/geometry.sage")
demo = False
from scipy.io import loadmat
import random

In [33]:
load("../framework/geometry.sage")
load("../framework/instance_gen.sage")


"""  Example
Uncomment the following to get an example
of the detailed computation (without redundancy)
"""
upper_bound_as_prob = True
to_print = True
# demo = True
# logging("--- Demonstration mode (no redundancy of the attacks) ---")

for params in [ 'CCS1', 'CCS2', 'CCS3', 'CCS4', 'NIST1', 'NIST2']:
    logging("Set of parameters: " + params)

    if params == 'NIST1':
        # NIST1 FRODOKEM-640
        n = 640
        m = 640
        q = 2**15
        frodo_distribution = [9456, 8857, 7280, 5249, 3321,
                              1844, 898, 384, 144, 47, 13, 3]
        D_s = get_distribution_from_table(frodo_distribution, 2 ** 16)
        # We used the following seeds for generating Bos et al. data
        # These seeds were generated with the matlab code genValues.m
        sca_seeds = [42, 72, 163, 175, 301, 320, 335, 406, 430, 445]
        param = 4

    elif params == 'NIST2':
        # NIST2 FRODOKEM-976
        n = 976
        m = 976
        q = 65536
        frodo_distribution = [11278, 10277, 7774, 4882, 2545, 1101,
                              396, 118, 29, 6, 1]
        D_s = get_distribution_from_table(frodo_distribution, 2 ** 16)
        sca_seeds = [74, 324, 337, 425, 543, 1595, 1707, 2026, 2075, 2707]
        param = 5

    elif params == 'CCS1':
        n = 352
        m = 352
        q = 2 ** 11
        frodo_distribution = [22528, 15616, 5120, 768]
        D_s = get_distribution_from_table(frodo_distribution, 2 ** 16)
        sca_seeds = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
        param = 0

    elif params == 'CCS2':
        n = 592
        m = 592
        q = 2 ** 12
        frodo_distribution = [25120, 15840, 3968, 384, 16]
        D_s = get_distribution_from_table(frodo_distribution, 2 ** 16)
        sca_seeds = [2, 3, 4, 5, 6, 7, 8, 9, 14, 16]
        param = 1


    elif params == 'CCS3':
        n = 752
        m = 752
        q = 2 ** 15
        frodo_distribution = [19296, 14704, 6496, 1664, 240, 16]
        D_s = get_distribution_from_table(frodo_distribution, 2 ** 16)
        sca_seeds = [2, 4, 5, 7, 9, 10, 12, 13, 14, 15]
        param = 2

    elif params == 'CCS4':
        n = 864
        m = 864
        q = 2 ** 15
        frodo_distribution = [19304, 14700, 6490, 1659, 245, 21, 1]
        D_s = get_distribution_from_table(frodo_distribution, 2 ** 16)
        sca_seeds = [256, 393, 509, 630, 637, 652, 665, 1202, 1264, 1387]
        param = 3


    """  Original Security   """

    A, b, dbdd = initialize_from_LWE_instance(DBDD_predict_diag,
                                               n,
                                               q, m, D_s, D_s, verbosity=0)
    dbdd.integrate_q_vectors(q, indices=range(n, n + m))
    (beta, _) = dbdd.estimate_attack()
    logging("Attack without hints:  %3.2f bikz" % beta, style="HEADER")

    """  Refined Side channel attack  """

    # Reading the score tables from Bos et al. attack
    scores = []
    correct = []
    for seed in sca_seeds:
        for i in range(1, 9):
            data = loadmat('Scores_tables_SCA/' + params +
                           '/apostandcorrect' + str(param + 1) +
                           '_seed' + str(seed) +
                           'nbar' + str(i) + '.mat')
            scores += list(data['apostdist'])
            correct += list(data['correct'])
    measures = {}
    # the score tables are stored according to the secret coefficient. We use them
    # for generating the measurement.
    for key_guess in range(-len(frodo_distribution)+1, len(frodo_distribution)):
        measures[key_guess] = [scores[ind] for ind in
                  [i for i, d in enumerate(correct) if
                   recenter(d) == key_guess]]


    if demo:
        A, b, dbdd = initialize_from_LWE_instance(DBDD_predict_diag,
                                                  n,
                                                  q, m, D_s, D_s, verbosity=2)
        measured = [simu_measured(dbdd.u[0, i], measures) for i in range(n)]
        if upper_bound_as_prob:
            estimate_SCA_prob(50, dbdd, measured, 0.5, D_s, D_s)
        else:
            estimate_SCA_num(50, dbdd, measured, 175, D_s, D_s)
    else:
        """  Averaging
        The following averages the measures to get accurate data
        for the paper. The averaging mode is quite long.
        """
        nb_tests_per_params = 2

        # Chosen values for the number of guesses
        if upper_bound_as_prob:
            max_guesses = 0.5
        else:
            if params == 'CCS1':
                max_guesses = 175 + 100
            elif params == 'CCS2':
                max_guesses = 300 + 150
            elif params == 'CCS3':
                max_guesses = 250 + 125
            elif params == 'CCS4':
                max_guesses = 250 + 125
            elif params == 'NIST1':
                max_guesses = 100 + 75
            elif params == 'NIST2':
                max_guesses = 250 + 125

        beta = 0
        beta_hybrid = 0
        proba_success = 0
        guesses_average = 0
        beta_hybrid_before_q = 0
        for i in range(nb_tests_per_params):
            A, b, dbdd = initialize_from_LWE_instance(DBDD_predict_diag,
                                                      n,
                                                      q, m, D_s, D_s, verbosity=0)
            measured = [simu_measured(dbdd.u[0, i], measures) for i in range(n)]
            if upper_bound_as_prob:
                b, b_hybrid, p_success, guesses, beta_before_q = estimate_SCA_prob(None, dbdd,
                                                  measured, max_guesses, D_s, to_print)
            else:
                b, b_hybrid, p_success, guesses = estimate_SCA_num(None, dbdd,
                                                  measured, max_guesses, D_s)
            beta += b
            beta_hybrid += b_hybrid
            proba_success += p_success
            guesses_average += guesses
            beta_hybrid_before_q += beta_before_q

        beta /= nb_tests_per_params
        beta_hybrid /= nb_tests_per_params
        proba_success /= nb_tests_per_params
        guesses_average /= nb_tests_per_params
        beta_hybrid_before_q /= nb_tests_per_params
        
        logging("Attack with hints: %3.2f bikz" % beta, style="HEADER")
        logging("Attack with hints and guess before q_vectors: %3.2f bikz"% beta_hybrid_before_q, style="HEADER")
        logging("Attack with hints and guess: %3.2f bikz"% beta_hybrid, style="HEADER")
        logging("Number of guesses: %4d" % guesses_average, style="HEADER")
        logging("Success probability: %3.2f" %proba_success, style="HEADER")


 Set of parameters: CCS1 
 Attack without hints:  268.83 bikz 
 order B: input1 SCA ellip, input2 LWE ellip 
rank 352 352 352 352
<class 'ValueError'>
f(a) and f(b) must have different signs
Using min/max as fallback...


/Applications/SageMath-9.1.app/Contents/Resources/sage/local/lib/python3.7/site-packages/scipy/optimize/_minimize.py:761: RuntimeWarning: Method 'bounded' does not support relative tolerance in x; defaulting to absolute tolerance.
  "defaulting to absolute tolerance.", RuntimeWarning)


poly(pi_sol) = 12.83078545303448
poly(1, 0) = 12.830782213153157 6211714230.660344
1.9267018696103588e-07
462
(Solution) pi = 0.999999962459470
alpha = 0.999999966171345
 order A: input1 LWE ellip, intput2 SCA ellip 
rank 352 352 352 352
<class 'ValueError'>
f(a) and f(b) must have different signs
Using min/max as fallback...
poly(pi_sol) = -12.830782977912861
poly(1, 0) = -6211714230.660344 -12.830782213153157
462
1.9267018696103588e-07
(Solution) pi = 8.86211698030706e-9
alpha = 0.999999992014182
 order B: input1 SCA ellip, input2 LWE ellip 
rank 352 352 352 352
156.21273467876608
462
(Solution) pi = 0.822879923661695
alpha = 0.831772037754149
 order A: input1 LWE ellip, intput2 SCA ellip 
rank 352 352 352 352
462
156.21273467876608
(Solution) pi = 0.177120076338502
alpha = 0.831772037753922
 Attack with hints: 164.87 bikz 
 Attack with hints and guess before q_vectors: 55.72 bikz 
 Attack with hints and guess: 37.81 bikz 
 Number of guesses:  249 
 Success probability: 0.51 
 Set of

In [25]:
def estimate_SCA_prob(report_every, dbdd, measured, max_guesses, D_s, to_print):
    #Upper bounding the maximum number of guessed coordinate
    """ 
    This function evaluates the security loss after Bos et al attack
    :report_every: an integer that give the frequency of
    logging (None for no logging)
    :dbdd: instance of the class DBDD
    :measured: table representing the (simulated) information
    given by Bos et al attack
    :max_guesses: integer for upperbounding the number of guesses
    """
    MINIMUM_VAR = 0.01
    proba_best = {}
    Id = identity_matrix(n + m)
    list_av = [0] * n
    list_var = [0] * n
    _, s_s = average_variance(D_s)
    intersect_indices = []
    for i in range(n):
        apost_dist = measurement_to_aposteriori(measured[i])
        if apost_dist is not None:
            proba_best[i] = max([apost_dist[j] for j in apost_dist.keys()])
            av, var = average_variance(apost_dist)
            av = float(av)
            var = float(var)
            list_av[i] = av
            list_var[i] = var
            if av*av > s_s - var:
                intersect_indices += [i]
        else:
            av = None
            var = None
        v = vec(Id[i])
        if report_every is not None and ((i % report_every == 0) or (i == n - 1)) :
            verbose = 2 
        else:
            verbose = 0
        dbdd.verbosity = verbose
        if verbose == 2:
            logging("[...%d]" % report_every, newline=False)
        if var is None:
            logging("   alert! var is None     ", style="HEADER")
        if var == 0:
            logging("     Real Perfect Hint     ", style="HEADER")
            list_var[i] = MINIMUM_VAR

    #####Intersection Phase####
#     dbdd.S, intersect_mu = reconstruct_diagonal_S(list_av, list_var, D_s, D_s, m, n, intersect_indices, to_print)
    dbdd.S, intersect_mu = reconstruct_diagonal_S(list_av, list_var, D_s, D_s, m, n, None, to_print)
   
    dbdd.mu = intersect_mu
    (beta, _) = dbdd.estimate_attack()

    
    #####Guess phase#####
    
    if report_every is not None:
        logging("     Hybrid attack estimation     ", style="HEADER")
    
    intersect_var = list(dbdd.S)
    intersect_av = list(dbdd.mu[0])
    proba_best2 = {}
    for k in range(len(intersect_var)):
        if var is None:
            logging("   alert! var is None     ", style="HEADER")
            continue
        if var == 0:
            v = vec(Id[k])
            logging("     Real Perfect Hint in Hybrid Phase    ", style="HEADER")
            intersect_var[k] = MINIMUM_VAR
            dbdd.S[k, k] = MINIMUM_VAR
            dbdd.integrate_perfect_hint(v, av, estimate=verbose)
        proba_best2[k] = var_to_prob(intersect_var[k], intersect_av[k], D_s, intersect_av[k])
          
    sorted_guesses = sorted(proba_best2.items(),
                            key=lambda kv: - kv[1])

    sorted_guesses = [sorted_guesses[i][0] for i in range(len(sorted_guesses))]

   
    proba_success = 1.
    dbdd.verbosity = 0
    guesses = 0
    j = 0
    
    for t in sorted_guesses:
        j += 1
        if proba_success*proba_best2[t] >= max_guesses: #(guesses <= max_guesses):(guesses <= max_guesses):

            v = vec(Id[t])
            if dbdd.integrate_perfect_hint(v, _, force = False):
                guesses += 1
                proba_success *= proba_best2[t]

    beta_before_last_q = dbdd.beta
    if report_every is not None:
        dbdd.integrate_q_vectors(q, min_dim = n + 1, indices=range(n, n + m), report_every=report_every)
    else:
        dbdd.integrate_q_vectors(q, min_dim = n + 1, indices=range(n, n + m))
    return beta, dbdd.beta, proba_success, guesses, beta_before_last_q


In [31]:
# def reconstruct_diagonal_S(list_av, list_var, D_s, D_e, m, n, intersect_indices, to_print):
##Selected indices
    
#     mu_e, s_e = average_variance(D_e)
#     mu_s, s_s = average_variance(D_s)
    
#     #ellipsoids of first n dimension, rescaling to <= 1
#     Sigma_SCA = diagonal_matrix(list_var)*n
#     mu_SCA = vec(list_av)
#     Sigma_LWE = matrix(Sigma_SCA)*n
#     mu_LWE = matrix(mu_SCA)
   
#     for i in intersect_indices:
#         Sigma_LWE[i, i] = s_s*n
#         mu_LWE[0, i] = mu_s
        
      
    
#     logging("order B: input1 SCA ellip, input2 LWE ellip", style="HEADER")
#     ellipsoid_intersection(mu_SCA , Sigma_SCA, mu_LWE, Sigma_LWE, tolerance=1.48e-08)
#     logging("order A: input1 LWE ellip, intput2 SCA ellip", style="HEADER")
#     new_mu_sub, new_S_sub = ellipsoid_intersection(mu_LWE, Sigma_LWE, mu_SCA , Sigma_SCA, tolerance=1.48e-08)
    
#     new_S1 = block4(diagonal_matrix([0] * m), zero_matrix(ZZ, m, n), zero_matrix(ZZ, n, m), new_S_sub)
#     new_S = block4(new_S1, zero_matrix(ZZ, n + m, 1), zero_matrix(ZZ, 1, n + m), zero_matrix(ZZ, 1, 1))
#     rest_S = diagonal_matrix(m * [s_e] + n * [0] + [1])
    
#     hybrid_S = (new_S/n + rest_S)
#     diagonal_S = np.array([RR(hybrid_S[i, i]) for i in range(hybrid_S.nrows())])
    
#     new_mu = concatenate([new_mu_sub, [mu_e]*m + [0]])
#     return diagonal_S, new_mu
def reconstruct_diagonal_S(list_av, list_var, D_s, D_e, m, n, intersect_indices, to_print):\
    ##full n submatrix
    mu_SCA = vec(list_av)
    S_SCA = diagonal_matrix(list_var)*(n)
    
    mu_e, s_e = average_variance(D_e)
    mu_s, s_s = average_variance(D_s)
    mu_LWE = vec(n * [mu_s])
    S_LWE = diagonal_matrix(n * [s_s])*(n)

    
    logging("order B: input1 SCA ellip, input2 LWE ellip", style="HEADER")
    ellipsoid_intersection(mu_SCA, S_SCA, mu_LWE, S_LWE, tolerance=1.48e-08)
    logging("order A: input1 LWE ellip, intput2 SCA ellip", style="HEADER")
    new_mu_sub, new_S_sub = ellipsoid_intersection(mu_LWE, S_LWE, mu_SCA, S_SCA, tolerance=1.48e-08)
    
    new_S1 = block4(diagonal_matrix([0] * m), zero_matrix(ZZ, m, n), zero_matrix(ZZ, n, m), new_S_sub)
    new_S = block4(new_S1, zero_matrix(ZZ, n + m, 1), zero_matrix(ZZ, 1, n + m), zero_matrix(ZZ, 1, 1))
    rest_S = diagonal_matrix(m * [s_e] + n * [0] + [1])
    
    hybrid_S = (new_S/n + rest_S)
    diagonal_S = np.array([RR(hybrid_S[i, i]) for i in range(hybrid_S.nrows())])
    
    new_mu = concatenate([new_mu_sub, [mu_e]*m + [0]])
    return diagonal_S, new_mu
def var_to_prob(var, mean, D_s, x):
    discrete_prob_list = [gaussian_prob(var, mean, x) for x in D_s.keys()]
#     print(discrete_prob_list)
    norm = sum(discrete_prob_list)

    denom = sqrt(var * 2 * pi)
    ex = exp(-0.5 * (x - mean)**2 / var)
    return ex/denom/norm
def gaussian_prob(var, mean, x):
    denom = sqrt(var * 2 * pi)
    ex = exp(-0.5 * (x - mean)**2 / var)
#     print(var, mean, x, denom, ex)
    return 1/denom * ex

In [8]:
def data_to_measure(sca_seeds, params, param, frodo_distribution): 
    # Reading the score tables from Bos et al. attack
    scores = []
    correct = []
    for seed in sca_seeds:
        for i in range(1, 9):
            data = loadmat('Scores_tables_SCA/' + params +
                           '/apostandcorrect' + str(param + 1) +
                           '_seed' + str(seed) +
                           'nbar' + str(i) + '.mat')
            scores += list(data['apostdist'])
            correct += list(data['correct'])
    measures = {}
    # the score tables are stored according to the secret coefficient. We use them
    # for generating the measurement.
    for key_guess in range(-len(frodo_distribution)+1, len(frodo_distribution)):
        measures[key_guess] = [scores[ind] for ind in
                  [i for i, d in enumerate(correct) if
                   recenter(d) == key_guess]]
    return measures
def simu_measured(secret, measures):
    """
    This fonction simulates the information gained by
    Bos et al attack. For a given secret, the simulation
    is outputting a score table at random from the data of Bos et al.
    :secret: an integer being the secret value
    :measurement: a score table
    """
    if secret in measures.keys():
        measurement = random.choice(measures[secret])
    else: 
        measurement = None
    return measurement

def measurement_to_aposteriori(measurement):
    """
    This fonction transforms a score table into an aposteriori distribution.
    According to the matlab code of Bos et al. attack,
    the score table is proportional
    to the logarithm of the aposteriori probability. We thus apply the exponential
    and renormalize.
    :measurement: a score table
    :apost_dist: a dictionnary of the aposteriori distribution
    """
    if measurement is not None:
        s = sum([exp(meas) for meas in measurement])
        measurement = [exp(meas)/s for meas in measurement]
        apost_dist = {}
        for key_guess in range(-len(frodo_distribution) + 1, len(frodo_distribution)):
            apost_dist[key_guess] = measurement[key_guess + len(frodo_distribution) - 1]
    else:
        apost_dist = None
    return apost_dist


In [27]:
#num
def estimate_SCA_num(report_every, dbdd, measured, max_guesses, D_s, to_print):
    #Upper bounding the maximum number of guessed coordinate
    """ 
    This function evaluates the security loss after Bos et al attack
    :report_every: an integer that give the frequency of
    logging (None for no logging)
    :dbdd: instance of the class DBDD
    :measured: table representing the (simulated) information
    given by Bos et al attack
    :max_guesses: integer for upperbounding the number of guesses
    """
    MINIMUM_VAR = 0.01
    proba_best = {}
    Id = identity_matrix(n + m)
    list_av = [0] * n
    list_var = [0] * n
    
    _, s_s = average_variance(D_s)
    intersect_indices = []
    
    for i in range(n):
        apost_dist = measurement_to_aposteriori(measured[i])
        if apost_dist is not None:
            proba_best[i] = max([apost_dist[j] for j in apost_dist.keys()])
            av, var = average_variance(apost_dist)
            av = float(av)
            var = float(var)
            list_av[i] = av
            list_var[i] = var
            if av*av > s_s - var:
                intersect_indices += [i]
        else:
            av = None
            var = None
        v = vec(Id[i])
        if report_every is not None and ((i % report_every == 0) or (i == n - 1)) :
            verbose = 2 
        else:
            verbose = 0
        dbdd.verbosity = verbose
        if verbose == 2:
            logging("[...%d]" % report_every, newline=False)
        if var is None:
            logging("   alert! var is None     ", style="HEADER")
        if var == 0:
            logging("     Real Perfect Hint     ", style="HEADER")
            list_var[i] = MINIMUM_VAR

    dbdd.S, intersect_mu = reconstruct_diagonal_S(list_av, list_var, D_s, D_s, m, n, None, to_print)
    dbdd.mu = intersect_mu
    
    (beta, _) = dbdd.estimate_attack()
    
    #####Guess phase#####
    if report_every is not None:
        logging("     Hybrid attack estimation     ", style="HEADER")
    
    intersect_var = list(dbdd.S) #only for DBDD_diag case
    intersect_av = list(intersect_mu[0])
    proba_best2 = {}
    for k in range(len(intersect_var)):

        if var is None:
            logging("   alert! var is None     ", style="HEADER")
            continue
        if var == 0:
            v = vec(Id[k])
            logging("     Nooo Real Perfect Hint     ", style="HEADER")

            intersect_var[k] = MINIMUM_VAR
            dbdd.S[k] = MINIMUM_VAR
            dbdd.integrate_perfect_hint(v, av, estimate=verbose)
        proba_best2[k] = var_to_prob(intersect_var[k], intersect_av[k], D_s, intersect_av[k])


    sorted_guesses = sorted(proba_best2.items(),
                            key=lambda kv: - kv[1])

    sorted_guesses = [sorted_guesses[i][0] for i in range(len(sorted_guesses))]

    proba_success = 1.
    dbdd.verbosity = 0
    guesses = 0
    j = 0
    for t in sorted_guesses:
        j += 1
        if (guesses < max_guesses):

            v = vec(Id[t])
            if dbdd.integrate_perfect_hint(v, _, force = False):
                guesses += 1
                proba_success *= proba_best2[t]
    beta_before_last_q = dbdd.beta
    print("before last q_vector", beta_before_last_q)
    if report_every is not None:
        dbdd.integrate_q_vectors(q, min_dim = n + 1, indices=range(n, n + m), report_every=report_every)
    else:
        dbdd.integrate_q_vectors(q, min_dim = n + 1, indices=range(n, n + m))
    return beta, dbdd.beta, proba_success, guesses, beta_before_last_q
